In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import os
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input
from tensorflow.keras.utils import to_categorical
from alibi.explainers import IntegratedGradients

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
%set_env CUDA_VISIBLE_DEVICES=2
tf.test.is_gpu_available()

env: CUDA_VISIBLE_DEVICES=2


True

In [4]:
tf.__version__

'1.15.0'

In [4]:
NUM_WORDS=1000 # only use top 1000 words
INDEX_FROM=3   # word index offset

word_to_id = tf.keras.datasets.imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
word_to_id["<UNUSED>"] = 3

id_to_word = {value:key for key,value in word_to_id.items()}

In [5]:
max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 100
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
y_train, y_test = to_categorical(y_train), to_categorical(y_test)
test_labels = np.argmax(y_test, axis=1)
train_labels = np.argmax(y_train, axis=1)
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)


Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 100)
x_test shape: (25000, 100)


In [6]:
print(' '.join(id_to_word[id] for id in x_train[10] ))

corner and this helps to ensure that maléfique actually does manage to be quite frightening the film is memorable for a lot of reasons outside the central plot the characters are all very interesting in their own way and the fact that the book itself almost takes on its own character is very well done anyone worried that the film won't deliver by the end won't be disappointed either as the ending both makes sense and manages to be quite horrifying overall maléfique is a truly great horror film and one of the best of the decade highly recommended viewing


# Model

In [7]:
load_model = True

In [8]:
filepath = './model_imdb/'  # change to directory where model is downloaded
if load_model:
    model = tf.keras.models.load_model(os.path.join(filepath, 'model.h5'))
else:
    print('Build model...')
    inputs = Input(shape=(x_train.shape[1:]), dtype=tf.float64)
    x = Embedding(max_features, 128)(inputs)
    x = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(x)
    outputs = Dense(2, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    # try using different optimizers and different optimizer configs
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    print('Train...')
    model.fit(x_train[:1000], y_train[:1000],
              batch_size=batch_size,
              epochs=1,
              validation_data=(x_test[:1000], y_test[:1000]))
    #score, acc = model.evaluate(x_test, y_test,
    #                            batch_size=batch_size)
    #print('Test score:', score)
    #print('Test accuracy:', acc)
        
    if not os.path.exists(filepath):
        os.makedirs(filepath)
    model.save(os.path.join(filepath, 'model.h5'))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
model.layers[1].output_shape

(None, 100, 128)

In [10]:
x_test[:1].shape

(1, 100)

In [11]:
n_steps = 5
method = "gausslegendre"
return_convergence_delta = True
return_predictions = False
ig  = IntegratedGradients(model, layer=model.layers[1],
                          n_steps=n_steps, 
                          method=method,
                          return_convergence_delta=return_convergence_delta, 
                          return_predictions=return_predictions)

In [12]:
nb_samples = 5
bs = 10
x_test_red = x_test[:nb_samples]
test_labels_red = test_labels[:nb_samples]

In [13]:
test_labels_red

array([0, 1, 1, 0, 1])

In [14]:
explanation = ig.explain(x_test_red, 
                         baselines=None, 
                         target=test_labels_red, 
                         internal_batch_size=bs)

In [15]:
attrs = explanation['data']['attributions']

The Explanation object is not a dictionary anymore and accessing elements should be done via attribute access. Accessing via item will stop working in a future version.


In [20]:
x_test_red[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     1,   591,   202,    14,
          31,     6,   717,    10,    10, 18142, 10698,     5,     4,
         360,     7,     4,   177,  5760,   394,   354,     4,   123,
           9,  1035,  1035,  1035,    10,    10,    13,    92,   124,
          89,   488,  7944,   100,    28,  1668,    14,    31,    23,
          27,  7479,    29,   220,   468,     8,   124,    14,   286,
         170,     8,   157,    46,     5,    27,   239,    16,   179,
       15387,    38,    32,    25,  7944,   451,   202,    14,     6,
         717], dtype=int32)

In [23]:
print(' '.join(id_to_word[id] for id in x_test_red[0] ))

<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <START> please give this one a miss br br kristy swanson and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite lacklustre so all you madison fans give this a miss


In [21]:
attrs_0 = attrs[0].sum(axis=1)

In [22]:
attrs_0

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -6.14434208e-09, -1.65919498e-08, -9.61821435e-09, -1.13566885e-08,
        2.18093225e-08, -1.63480299e-09,  1.72306335e-08,  7.04142433e-08,
        1.02810079e-07,  2.72824293e-07,  1.04251637e-07, -8.46694389e-08,
       -5.26614775e-07,  5.16491359e-07, -1.75367101e-07, -9.60274328e-07,
       -9.76610996e-08,  7.23789210e-07,  2.20366624e-06, -6.61745160e-07,
       -2.51425975e-06,  

In [66]:
sentence = ' '.join(id_to_word[id] for id in x_test_red[0])

In [64]:
word_to_id['<PAD>'] = 0

In [67]:
inp = [word_to_id[w] for w in sentence.split()]
inp = [0 for i in range(100 - len(inp))] + inp
inp = np.asarray(inp).reshape(1, -1)

In [68]:
pred = np.argmax(model(inp).numpy(), axis=1)
pred

array([1])

In [69]:
attrs = ig.explain(inp, target=pred)['data']['attributions']

The Explanation object is not a dictionary anymore and accessing elements should be done via attribute access. Accessing via item will stop working in a future version.


In [70]:
attrs[0].sum(axis=1)

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        6.14434890e-09,  1.65919591e-08,  9.61821494e-09,  1.13566703e-08,
       -2.18093007e-08,  1.63481875e-09, -1.72306381e-08, -7.04142481e-08,
       -1.02810082e-07, -2.72824276e-07, -1.04251585e-07,  8.46695051e-08,
        5.26614861e-07, -5.16491436e-07,  1.75367173e-07,  9.60274351e-07,
        9.76611269e-08, -7.23789080e-07, -2.20366611e-06,  6.61745087e-07,
        2.51425956e-06, -